# MiniProyecto Parte 3
Ana Lucía Hernández - 17138
María Fernanda López - 17160

In [69]:
import numpy as np

In [91]:
class Server():
    def __init__(self, potencia, max_sol = 2400, cpus = 1):
        # potencia es la cantidad de solicitudes por segundo que puede tomar 
        # max_sol es la cantidad máxima de solicitudes que se recibirán * por minuto * (lambda_max)
        # cpus es la cantidad de servidores que tiene el sistema 
        self.potencia = potencia
        self.lambda_max = max_sol  # lambda se toma como solicitudes/min (si no se tarda mucho)
        self.cpus = cpus
    
    def next_ts(self, t): # los eventos serán solamente procesos de poisson homogeneos, así que se programa solo para eso 
        return t - (np.log(np.random.uniform())/self.lambda_max)

    def get_exponential(self, lamda):
        return -(1 / lamda)*np.log(np.random.uniform())

    def simulate(self):
        # se asumirá que t0 = 0 y T = t0 + 60min 
        t = 0 
        n = 0 # estado del sistema, numero de solicitudes en el sistema al tiempo t 
        T = 60

        # contadores
        Na = 0 # llegadas 
        Nd = 0 # salidas

        A = [] # tiempos de llegada de la i-esima solicitud, ids son indices
        D = [] # tiempos de salida de la i-esima solicitud, ids son indices
        NT = [] # Tiempos de cada cliente en espera

        # eventos
        ta = self.next_ts(t) # tiempo de la proxima llegada
        td = np.zeros(self.cpus) + np.infty # set de tiempos de salida de cada servidor a infty, hay un td por cada server disponible
        busy_time = np.zeros(self.cpus) # tiempo que cada server estuvo ocupado
        served_by = [] # se guardan cuales solicitudes fueron atendidas por cuales server
        servers = np.zeros(self.cpus) # para llevar registro de quien esta ocupado y quien no

        while t < T: # el tiempo en el que estamos aún no excede el tiempo de cierre
            # CASO 1 
            # si el proximo tiempo de llegada es antes del proximo tiempo de salida, se encola
            if ta < min(td):
                t = ta # nos posicionamos en el próximo tiempo de llegada
                Na += 1 # Contamos una llegada mas
                ta = self.next_ts(t) # calculamos el siguiente tiempo de llegada
                A.append(t) # guardamos el tiempo de la Na-esima llegada
                if n < self.cpus: # si hay menos clientes dentro que servidores, se le asigna uno que esté disponible
                    for i in range(self.cpus):
                        if servers[i] == 0: # Si el servidor esta disponible
                            NT = np.append(NT,t - A[len(A)-1])
                            served_by.append(i) # se agrega el server que atendio el # de solicitud
                            td[i] = t + self.get_exponential(1/(self.potencia*60)) # calculamos su tiempo de salida y se lo asignamos a un servidor (segundos por solicitud)
                            busy_time[i] += td[i]-t # Calculamos el tiempo que va a estar en el servidor
                            servers[i] = 1 # seteamos el server como ocupado 
                            break;
                n += 1 # Contamos al nuevo cliente en el sistema
            
            # CASO 2
            # si el proximo tiempo de llegada es después del próximo tiempo de salida, se atiende ya que
            # se tiene la disponibilidad
            else:
                #if np.infty not in td else list(td).index(np.infty)
                ti = np.argmin(td) # servidor que terminará más próximamente
                t = td[ti] # avanzamos el tiempo al tiempo en el que termina
                D.append(t)
                if n <= self.cpus: # Si hay menos o igual clientes que servidores
                    servers[ti] = 0 # liberamos el server
                    td[ti] = np.infty # seteamos el td a infty para indicar que aun no tiene otra solicitud
                else: # Si todavia hay mas clientes esperando

                    served_by.append(ti) # se agrega el server que atendio el # de solicitud
                    NT = np.append(NT,t - A[len(A)-1])
                    td[ti] = t + self.get_exponential(1/(self.potencia*60)) # Calculamos el tiempo de salida
                    busy_time[ti] += td[ti]-t # Calculamos el tiempo que va a estar en el servidor
                    servers[i] = 1 # seteamos el server como ocupado 
                n -= 1 # Descontamos al cliente atendido del sistema
        # se calcula cuantas solicitudes atendio cada servidor 
        num_sol = np.zeros(self.cpus)
        for i in range(len(served_by)):
            num_sol[served_by[i]] += 1

        return { 
            "en_cola": NT, "num_sol": num_sol, "td": td, "A": A, "D": D, "busy_time": busy_time
        }

## Task 1 - Gorilla Megacomputing, 2400 max solicitudes

In [92]:
gorilla = Server(potencia = 100)
resultados = gorilla.simulate()

In [93]:
print("1. ¿Cuántas solicitudes atendió cada servidor?")
print("\tSolicitudes atendidas por servidor: ",resultados["num_sol"][0])
print("\n2. ¿Cuánto tiempo estuvo cada servidor ocupado?")
print("\tTiempos de ocupacion de servidores: ", resultados["busy_time"][0])
print("\n3. ¿Cuánto tiempo estuvo cada servidor desocupado (idle)?")
print("\tTiempo en el que el servidor estuvo libre: ",np.maximum(np.ones(gorilla.cpus)*60 - resultados["busy_time"],0)[0])
print("\n4. Cuánto tiempo en total estuvieron las solicitudes en cola?")
print("\tTiempo total de solicitudes en cola: ",np.round(sum(resultados["en_cola"]),5))
print("\n5. En promedio ¿cuánto tiempo estuvo cada solicitud en cola?")
print("\tTiempo promedio de solicitudes en cola:",np.round(np.mean(resultados["en_cola"]),5))
print("\n6. En promedio, ¿cuántas solicitudes estuvieron en cola cada segundo?")
sol_psec = [ 1/num if num != 0 else 0 for num in resultados["en_cola"] ]
print("\tSolicitudes promedio en cola por segundo:",np.round(np.mean(sol_psec),5))
print("\n7. ¿Cuál es el momento de la salida de la última solicitud?")
print("\tTiempo de salida de la ultima solicitud atendida: ",np.round(resultados["td"][-1],5), "min") 

1. ¿Cuántas solicitudes atendió cada servidor?
	Solicitudes atendidas por servidor:  144216.0

2. ¿Cuánto tiempo estuvo cada servidor ocupado?
	Tiempos de ocupacion de servidores:  24.042495589453324

3. ¿Cuánto tiempo estuvo cada servidor desocupado (idle)?
	Tiempo en el que el servidor estuvo libre:  35.957504410546676

4. Cuánto tiempo en total estuvieron las solicitudes en cola?
	Tiempo total de solicitudes en cola:  9.65594

5. En promedio ¿cuánto tiempo estuvo cada solicitud en cola?
	Tiempo promedio de solicitudes en cola: 7e-05

6. En promedio, ¿cuántas solicitudes estuvieron en cola cada segundo?
	Solicitudes promedio en cola por segundo: 30561.53495

7. ¿Cuál es el momento de la salida de la última solicitud?
	Tiempo de salida de la ultima solicitud atendida:  60.00058 min


## Task 1 - Ants Computing, 2400 max solicitudes

In [76]:
ant = Server(potencia=10, cpus = 10)
resultados_ac = ant.simulate()

In [81]:
print("1. ¿Cuántas solicitudes atendió cada servidor?")
print("\tSolicitudes atendidas por servidor: ", "".join([ "\n\t\tServidor "+str(i)+": "+str(np.round(resultados_ac["num_sol"][i], 5)) for i in range(len(resultados_ac["num_sol"])) ]))
print("\n2. ¿Cuánto tiempo estuvo cada servidor ocupado?")
print("\tTiempos de ocupacion de servidores: ", "".join([ "\n\t\tServidor "+str(i)+": "+str(np.round(resultados_ac["busy_time"][i], 5)) for i in range(len(resultados_ac["busy_time"])) ]))
print("\n3. ¿Cuánto tiempo estuvo cada servidor desocupado (idle)?")
idle = np.maximum(np.ones(gorilla.cpus)*60 - resultados_ac["busy_time"],0)
print("\tTiempo en el que el servidor estuvo libre: ", "".join([ "\n\t\tServidor "+str(i)+": "+str(idle[i]) for i in range(len(idle)) ]))
print("\n4. Cuánto tiempo en total estuvieron las solicitudes en cola?")
print("\tTiempo total de solicitudes en cola: ",np.round(sum(resultados_ac["en_cola"]),5))
print("\n5. En promedio ¿cuánto tiempo estuvo cada solicitud en cola?")
print("\tTiempo promedio de solicitudes en cola:",np.round(np.mean(resultados_ac["en_cola"]),5))
print("\n6. En promedio, ¿cuántas solicitudes estuvieron en cola cada segundo?")
sol_psec = [ 1/num if num != 0 else 0 for num in resultados_ac["en_cola"] ]
print("\tSolicitudes promedio en cola por segundo:",np.round(np.mean(sol_psec),5))
print("\n7. ¿Cuál es el momento de la salida de la última solicitud?")
print("\tTiempo de salida de la ultima solicitud atendida: ",np.round(resultados_ac["td"][-1],5), "min") 

1. ¿Cuántas solicitudes atendió cada servidor?
	Solicitudes atendidas por servidor:  
		Servidor 0: 28802.0
		Servidor 1: 26640.0
		Servidor 2: 23670.0
		Servidor 3: 20428.0
		Servidor 4: 16311.0
		Servidor 5: 11928.0
		Servidor 6: 8007.0
		Servidor 7: 4849.0
		Servidor 8: 2581.0
		Servidor 9: 1267.0

2. ¿Cuánto tiempo estuvo cada servidor ocupado?
	Tiempos de ocupacion de servidores:  
		Servidor 0: 48.11639
		Servidor 1: 44.45916
		Servidor 2: 39.74967
		Servidor 3: 33.98147
		Servidor 4: 27.17337
		Servidor 5: 20.19584
		Servidor 6: 13.39329
		Servidor 7: 7.98213
		Servidor 8: 4.29817
		Servidor 9: 2.15083

3. ¿Cuánto tiempo estuvo cada servidor desocupado (idle)?
	Tiempo en el que el servidor estuvo libre:  
		Servidor 0: 11.883605446353023
		Servidor 1: 15.540842772885696
		Servidor 2: 20.25032726849431
		Servidor 3: 26.01853141299094
		Servidor 4: 32.8266273397346
		Servidor 5: 39.8041573559106
		Servidor 6: 46.606714525128545
		Servidor 7: 52.01786648371677
		Servidor 8: 55.7018

## Task 2 - 2400 max solicitudes

In [113]:
ant = Server(potencia=10, cpus = 12)
resultados12 = ant.simulate()

In [114]:
print("\tTiempo total de solicitudes en cola con 12 cpus: ",np.round(sum(resultados12["en_cola"]),5))

	Tiempo total de solicitudes en cola con 12 cpus:  0.0222


In [115]:
ant = Server(potencia=10, cpus = 15)
resultados15 = ant.simulate()

In [116]:
print("\tTiempo total de solicitudes en cola con 15 cpus: ",np.round(sum(resultados15["en_cola"]),5))

	Tiempo total de solicitudes en cola con 15 cpus:  5e-05


In [117]:
ant = Server(potencia=10, cpus = 16)
resultados16 = ant.simulate()

In [118]:
print("\tTiempo total de solicitudes en cola con 16 cpus: ",np.round(sum(resultados16["en_cola"]),5))

	Tiempo total de solicitudes en cola con 16 cpus:  0.0


Podemos observar en las simulaciones que a partir de 16 cpus ya contamos con un total de tiempo en cola de 0, lo que indica que ninguna solicitud esperó en cola. 

## Task 3 - Gorilla Megacomputing, 6000 max solicitudes

In [87]:
gorilla = Server(potencia = 100, max_sol = 6000)
resultados_g2 = gorilla.simulate()

In [119]:
print("1. ¿Cuántas solicitudes atendió cada servidor?")
print("\tSolicitudes atendidas por servidor: ",resultados_g2["num_sol"][0])
print("\n2. ¿Cuánto tiempo estuvo cada servidor ocupado?")
print("\tTiempos de ocupacion de servidores: ", resultados_g2["busy_time"][0])
print("\n3. ¿Cuánto tiempo estuvo cada servidor desocupado (idle)?")
print("\tTiempo en el que el servidor estuvo libre: ",np.maximum(np.ones(gorilla.cpus)*60 - resultados_g2["busy_time"],0)[0])
print("\n4. Cuánto tiempo en total estuvieron las solicitudes en cola?")
print("\tTiempo total de solicitudes en cola: ",np.round(sum(resultados_g2["en_cola"]),5))
print("\n5. En promedio ¿cuánto tiempo estuvo cada solicitud en cola?")
print("\tTiempo promedio de solicitudes en cola:",np.round(np.mean(resultados_g2["en_cola"]),5))
print("\n6. En promedio, ¿cuántas solicitudes estuvieron en cola cada segundo?")
sol_psec = [ 1/num if num != 0 else 0 for num in resultados_g2["en_cola"] ]
print("\tSolicitudes promedio en cola por segundo:",np.round(np.mean(sol_psec),5))
print("\n7. ¿Cuál es el momento de la salida de la última solicitud?")
print("\tTiempo de salida de la ultima solicitud atendida: ",np.round(resultados_g2["td"][-1],5), "min") 

1. ¿Cuántas solicitudes atendió cada servidor?
	Solicitudes atendidas por servidor:  359189.0

2. ¿Cuánto tiempo estuvo cada servidor ocupado?
	Tiempos de ocupacion de servidores:  59.99868028495467

3. ¿Cuánto tiempo estuvo cada servidor desocupado (idle)?
	Tiempo en el que el servidor estuvo libre:  0.001319715045326575

4. Cuánto tiempo en total estuvieron las solicitudes en cola?
	Tiempo total de solicitudes en cola:  60.01529

5. En promedio ¿cuánto tiempo estuvo cada solicitud en cola?
	Tiempo promedio de solicitudes en cola: 0.00017

6. En promedio, ¿cuántas solicitudes estuvieron en cola cada segundo?
	Solicitudes promedio en cola por segundo: 144917.93068

7. ¿Cuál es el momento de la salida de la última solicitud?
	Tiempo de salida de la ultima solicitud atendida:  60.00018 min


## Task 3 - Ants Computing, 6000 max solicitudes

In [120]:
ant = Server(potencia = 10, max_sol = 6000, cpus=10)
ant6k = ant.simulate()

In [121]:
print("1. ¿Cuántas solicitudes atendió cada servidor?")
print("\tSolicitudes atendidas por servidor: ", "".join([ "\n\t\tServidor "+str(i)+": "+str(np.round(ant6k["num_sol"][i], 5)) for i in range(len(ant6k["num_sol"])) ]))
print("\n2. ¿Cuánto tiempo estuvo cada servidor ocupado?")
print("\tTiempos de ocupacion de servidores: ", "".join([ "\n\t\tServidor "+str(i)+": "+str(np.round(ant6k["busy_time"][i], 5)) for i in range(len(ant6k["busy_time"])) ]))
print("\n3. ¿Cuánto tiempo estuvo cada servidor desocupado (idle)?")
idle = np.maximum(np.ones(gorilla.cpus)*60 - ant6k["busy_time"],0)
print("\tTiempo en el que el servidor estuvo libre: ", "".join([ "\n\t\tServidor "+str(i)+": "+str(idle[i]) for i in range(len(idle)) ]))
print("\n4. Cuánto tiempo en total estuvieron las solicitudes en cola?")
print("\tTiempo total de solicitudes en cola: ",np.round(sum(ant6k["en_cola"]),5))
print("\n5. En promedio ¿cuánto tiempo estuvo cada solicitud en cola?")
print("\tTiempo promedio de solicitudes en cola:",np.round(np.mean(ant6k["en_cola"]),5))
print("\n6. En promedio, ¿cuántas solicitudes estuvieron en cola cada segundo?")
sol_psec = [ 1/num if num != 0 else 0 for num in ant6k["en_cola"] ]
print("\tSolicitudes promedio en cola por segundo:",np.round(np.mean(sol_psec),5))
print("\n7. ¿Cuál es el momento de la salida de la última solicitud?")
print("\tTiempo de salida de la ultima solicitud atendida: ",np.round(ant6k["td"][-1],5), "min") 

1. ¿Cuántas solicitudes atendió cada servidor?
	Solicitudes atendidas por servidor:  
		Servidor 0: 36015.0
		Servidor 1: 35992.0
		Servidor 2: 35962.0
		Servidor 3: 35875.0
		Servidor 4: 35982.0
		Servidor 5: 35844.0
		Servidor 6: 36167.0
		Servidor 7: 36026.0
		Servidor 8: 35851.0
		Servidor 9: 35790.0

2. ¿Cuánto tiempo estuvo cada servidor ocupado?
	Tiempos de ocupacion de servidores:  
		Servidor 0: 59.9874
		Servidor 1: 59.97895
		Servidor 2: 59.97671
		Servidor 3: 59.97361
		Servidor 4: 59.97111
		Servidor 5: 59.96372
		Servidor 6: 59.95549
		Servidor 7: 59.95351
		Servidor 8: 59.93927
		Servidor 9: 59.91503

3. ¿Cuánto tiempo estuvo cada servidor desocupado (idle)?
	Tiempo en el que el servidor estuvo libre:  
		Servidor 0: 0.012601367809672581
		Servidor 1: 0.02105190809953683
		Servidor 2: 0.02329219707445418
		Servidor 3: 0.026394131454701153
		Servidor 4: 0.028887800123818863
		Servidor 5: 0.03628198297762708
		Servidor 6: 0.044510852531509215
		Servidor 7: 0.04648573764988

## Task 4 - 6000 max solicitudes

In [130]:
ant = Server(potencia=10, cpus = 16, max_sol = 6000)
resultados16 = ant.simulate()

In [131]:
print("\tTiempo total de solicitudes en cola con 16 cpus: ",np.round(sum(resultados16["en_cola"]),5))

	Tiempo total de solicitudes en cola con 16 cpus:  2.16651


In [144]:
ant = Server(potencia=10, cpus = 27, max_sol = 6000)
resultados27 = ant.simulate()

In [145]:
print("\tTiempo total de solicitudes en cola con 27 cpus: ",np.round(sum(resultados27["en_cola"]),5))

	Tiempo total de solicitudes en cola con 27 cpus:  0.0001


In [140]:
ant = Server(potencia=10, cpus = 28, max_sol = 6000)
resultados28 = ant.simulate()

In [141]:
print("\tTiempo total de solicitudes en cola con 28 cpus: ",np.round(sum(resultados28["en_cola"]),5))

	Tiempo total de solicitudes en cola con 28 cpus:  0.0


Podemos observar que con 6000 solicitudes como máximo, el numero de cpus tendría que aumentar a un mínimo de 28 para que ningún proceso se quede en cola esperando.

## Task 5

Se puede observar que para 2400 solicitudes que son las que según el departamento serian las maximas a las que se puede llegar, Ants Computing logra atender una mayor cantidad de solicitudes dentro de la hora en que se simuló, además que se mantienen más tiempo ocupados - en promedio - lo que indica que se toma más provecho de los recursos; además que tienen menores tiempos de espera. Por tanto, la recomendación es que alquilen los 10 servidores. 